## AutoEncoders on PointClouds - Modular Code

In [1]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet

from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.point_net_ae import Configuration as PN_Conf

# import tf_lab.point_clouds.various_encoders_decoders as enc_dec
# import tf_lab.models.point_net_based_AE as pnAE

from general_tools.in_out import create_dir
from geo_tool import Point_Cloud

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

seed = 42
np.random.seed(seed)
tf.set_random_seed(seed)

In [3]:
data_path = '/Users/optas/DATA/Point_Clouds/Shape_Net_Core/from_manifold_meshes/1024/03001627'
# LOG_PATH = '/Users/optas/DATA/Neural_Nets/Train_Log/Point_Cloud_AE/'
# create_dir(LOG_PATH)

train_dir = '/Users/optas/DATA/Neural_Nets/Models/Point_Cloud_AE/'
experiment_name = 'test_denoising_ae'

model_path = osp.join(train_dir, experiment_name)
create_dir(model_path)

'/Users/optas/DATA/Neural_Nets/Models/Point_Cloud_AE/test_denoising_ae'

In [4]:
file_names = pio.load_filenames_of_input_data(data_path)
file_names = file_names[:1000]
all_pclouds, model_names = pio.load_crude_point_clouds(file_names=file_names, n_threads=11)


train_data_, val_data_, test_data_ = pio.train_validate_test_split(all_pclouds, 
                                                                   train_perc=0.8, 
                                                                   validate_perc=0.1, 
                                                                   test_perc=0.1, 
                                                                   seed=seed)

# train_data = PointCloudDataSet(train_data_)
# val_data = PointCloudDataSet(val_data_)
# test_data = PointCloudDataSet(test_data_)

train_data = PointCloudDataSet(train_data_, noise={'frac':0.20, 'filler':0.0})
val_data = PointCloudDataSet(val_data_, noise={'frac':0.20, 'filler':0.0})
test_data = PointCloudDataSet(test_data_, noise={'frac':0.20, 'filler':0.0})

6778 files containing  point clouds were found.


In [5]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()


conf = PN_Conf(n_input = [1024, 3],
               training_epochs = 16,
               batch_size = 20,
               loss = 'l2',
               train_dir = train_dir,
               loss_display_step = 1,               
               saver_step = 2,
               saver_max_to_keep = 5,
               learning_rate=0.0002,
               denoising=True
              )

reset_graph()
name = 'test_denoising_ae'
ae = PointNetAutoEncoder(name, conf)
ae.train(train_data, conf)

('Epoch:', '0001', 'training time (minutes)=', '0.4756', 'loss=', '0.018331316')
('Epoch:', '0002', 'training time (minutes)=', '0.4772', 'loss=', '0.015801058')
('Epoch:', '0003', 'training time (minutes)=', '0.4788', 'loss=', '0.015693301')
('Epoch:', '0004', 'training time (minutes)=', '0.4798', 'loss=', '0.015588190')
('Epoch:', '0005', 'training time (minutes)=', '0.4765', 'loss=', '0.015473706')
('Epoch:', '0006', 'training time (minutes)=', '0.4822', 'loss=', '0.015352813')
('Epoch:', '0007', 'training time (minutes)=', '0.4781', 'loss=', '0.015262323')
('Epoch:', '0008', 'training time (minutes)=', '0.4752', 'loss=', '0.015205709')
('Epoch:', '0009', 'training time (minutes)=', '0.4737', 'loss=', '0.015160337')


KeyboardInterrupt: 

In [7]:
reset_graph()
name = 'test_ae'
ae = PointNetAutoEncoder(name, conf)
ae.restore_model(conf.train_dir, 4)
train_data = PointCloudDataSet(train_data_)
# ae.train(train_data, conf)

Model restored in epoch 4.
